In [1]:
import requests
from bs4 import BeautifulSoup
import pandas, os, numpy, re, traceback, os

台北市
https://www.104.com.tw/jobs/search/?ro=0&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&「area=6001001000」&order=16&asc=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1

台北市中正區
https://www.104.com.tw/jobs/search/?ro=0&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&「area=6001001001」&order=16&asc=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1

台北市中正區第一個職務類別
https://www.104.com.tw/jobs/search/?ro=0&「jobcat=2001000000」&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&「area=6001001001」&order=16&asc=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1

台北市中正區第三個職務類別
https://www.104.com.tw/jobs/search/?ro=0&「jobcat=2003000000」&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&「area=6001001001」&order=16&asc=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1

https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=6001023000&order=16&asc=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1

# 備註 
每次會跑出150頁，每頁20-30組資料。換言之，不能以一個地區去爬取，游騰林是使用關鍵字去爬取。

In [1]:
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

In [2]:
for area_district in range(1, 1) : 
    print(area_district)
    print("c")

In [3]:
result_list = []
error_list = []
url_base = "https://www.104.com.tw/jobs/search/?ro=1" # 僅找全職人員，遺漏的資料較少。
position_classfication = r"&jobcat=20{職業編號}000000&" # 共有18個職務類別，格式 01
area_classfication = r"&area=60010{縣市編號}0{區域編號}" # 共有 23個縣市，格式 01, 01
area_district_dict = {1:12, 2:29, 3:12, 4:7, 5:13, 6:14, 7:18, 8:29, 10:26, 11:13, 12:20, 13:19, 14:37, 16:38, 18:33, 19:16, 20:6, 21:6, 22:6}
# 104 : 9,15,17 跳號。23號連江，另外處理。

# X 市
for area_county in range(1, 23) : 
    if area_county not in area_district_dict.keys() : 
        continue

    area_county_str = str(area_county) 
    print(f"第{area_county}個縣市")
    if area_county < 10 : 
        area_county_str = "0" + area_county_str

    for area_position in range(1, 19) : 
        print(f"第{area_position}個職業")
        area_position_str = str(area_position)
        if area_position < 10 :
            area_position_str = "0" + area_position_str
 
        for area_district in range(1, area_district_dict[area_county] + 1) : # 沒有該縣市沒有區，只有縣市，那得另外處理。
            print(f"第{area_district}區，共{area_district_dict[area_county]}區")
            area_district_str = str(area_district)
            if area_district < 10 :
                area_district_str = "0" + area_district_str

            # 這裡要使用 if 來分流。字串會寫入錯誤。
            url_area_county_position = url_base + f"&jobcat=20{area_position_str}000000&&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&area=60010{area_county_str}0{area_district_str}&order=16&asc=0"

            # X 市 X 區 X 職務之類別，所具有的頁面搜尋。最多只有150頁。
            try : 
                for page in range(1, 151) : 
                    url_continue = url_area_county_position + f"&page={page}&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1"
                    url_continue_html = requests.get(url_continue)

                    # 如果 area_district 的編號，不存在於該縣市中，則 104 會回傳全部工作資訊回來，此時不要爬取，直接 break。
                    soup = BeautifulSoup(url_continue_html.text, "html.parser")
                    # 設定上限的方法會因為 javaScript 的回傳而無效，那就自己做字典，讓爬取不會超過區域數。

                    articles = soup.find_all("article")

                    # 該頁之20個工作資訊爬取，並儲存。
                    for article in articles :  
                        # company_url、work_name
                        try :         
                            token_one = article.find("h2")
                            company_url = token_one.find("a")["href"] # company_url
                        except : 
                            company_url = numpy.nan
                        
                        try : 
                            token_one = article.find("h2").find("a")
                            work_name = token_one.text # work_name
                        except : 
                            work_name = numpy.nan

                        # company_name、industry
                        # industry
                        try : 
                            token_two = article.find("ul", class_ = "b-list-inline b-clearfix")
                            token_li_s = token_two.find_all("li")
                            industry = token_li_s[2].text
                        except : 
                            industry = numpy.nan
                        
                        # company_name
                        try : 
                            token_two = article.find("ul", class_ = "b-list-inline b-clearfix")
                            token_li_s = token_two.find_all("li")
                            company_name =  token_li_s[1].find("a")["title"]
                        except : 
                            company_name = numpy.nan
                        
                        # area、seniority、education
                        # area
                        try : 
                            token_three = article.find("ul", class_ = "b-list-inline b-clearfix job-list-intro b-content")
                            token_li_s = token_three.find_all("li")
                            area = token_li_s[0].text
                        except : 
                            area = numpy.nan
                        
                        # seniority
                        try : 
                            token_three = article.find("ul", class_ = "b-list-inline b-clearfix job-list-intro b-content")
                            token_li_s = token_three.find_all("li")
                            seniority = token_li_s[1].text
                        except : 
                            seniority = numpy.nan

                        # education
                        try : 
                            token_three = article.find("ul", class_ = "b-list-inline b-clearfix job-list-intro b-content")
                            token_li_s = token_three.find_all("li")
                            education = token_li_s[2].text
                        except : 
                            education = numpy.nan

                        # number_of_applicants
                        try : 
                            token_four = article.find("div", class_ = "b-block__right b-pos-relative")
                            number_of_applicants = token_four.find("a", class_ = "b-link--gray gtm-list-apply").text
                        
                        except : 
                            number_of_applicants = numpy.nan

                        # salary
                        try : 
                            token_five = article.find("div", class_ = "job-list-tag b-content")
                            salary = token_five.find_all("a", class_ = "b-tag--default")[0].text
                        except : 
                            salary = numpy.nan
                            
                        # company_size
                        try : 
                            token_five = article.find("div", class_ = "job-list-tag b-content")
                            company_size = token_five.find_all("a", class_ = "b-tag--default")[1].text
                        except : 
                            company_size = numpy.nan
                        
                        # requirements
                        try : 
                            token_six = article.find("p", class_ = "job-list-item__info")
                            requirements = token_six.text
                        except : 
                            requirements = numpy.nan 

                        result = {"Profession_classification" : area_position, "company_name" : company_name, "work_name" : work_name, "requirements" : requirements, 
                                "104_url" : company_url, "area" : area, "education" : education, "industry" : industry, 
                                "seniority" : seniority, "number_of_applicants" : number_of_applicants, 
                                "salary" : salary, "company_size" : company_size}
                        result_list.append(result)
                        
                    
                    # 每一個分頁最多只有20個工作顯示以及兩個空的欄位，換言之，當出現任何一頁的標籤數小於22，則意謂著沒有下一頁可以爬取。
                    if len(articles) < 22 : 
                        break
            except : 
                error_elemnt = {"location" : f"報錯，第{area_district}區，共{area_district_dict[area_county]}區", 
                                "python" : traceback.format_exc()}
                error_list.append(error_elemnt)
                continue

result_df = pandas.DataFrame(result_list)
error_df = pandas.DataFrame(error_list)

# 資料清理(numpy.nan)
Hualien_df = result_df.copy()
Hualien_df = Hualien_df.dropna(how= "all")
# 抓出 salary 為遺漏值的行
Hualien_df["salary"] = Hualien_df["salary"].fillna("待遇面議")
Hualien_df.info()



第1個縣市
第1個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第2個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第3個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第4個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第5個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第6個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第7個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第8個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12區
第7區，共12區
第8區，共12區
第9區，共12區
第10區，共12區
第11區，共12區
第12區，共12區
第9個職業
第1區，共12區
第2區，共12區
第3區，共12區
第4區，共12區
第5區，共12區
第6區，共12

In [4]:
Hualien_df.head(3)

,Profession_classification,company_name,work_name,requirements,104_url,area,education,industry,seniority,number_of_applicants,salary,company_size
0,1,公司名：【企業專案招募】104未來無限整合招募中心\n公司住址：台北市中正區衡陽路(西門捷運...,【找對工作 成就財富自由人生】 加入我們翻轉你的人生_電話行銷專員(台北)【企業專案招募】,✦✧✦安達國際人壽保險股份有限公司✦✧✦\n\n【公司介紹】\n◆ 全球最大的產物保險與意外...,//www.104.com.tw/job/7viq3?jobsource=hotjob_chr,台北市中正區,高中,NaN,經歷不拘,0~5人應徵,"月薪27,000~100,000元",NaN
1,1,公司名：全家國際餐飲股份有限公司\n公司住址：台北市中正區北平西路3號2樓(東3門),大戶屋日本定食料理[台北車站店]- 內場正職,~~一定就是你了~~\n\n從基層服務或料理製作開始學習起，享有健全之教育訓練體制，暢通之升...,//www.104.com.tw/job/59fcp?jobsource=hotjob_chr,台北市中正區,高中,NaN,經歷不拘,0~5人應徵,"月薪34,000~38,000元",員工1000人
2,1,公司名：楽坡BonBox_樂邁喬新商行\n公司住址：台北市中正區八德路一段82巷12號,【正職 / 兼職皆可】無油煙健康餐盒 門市廚房及服務人員 ( 每月都會發團體達標獎金！),我們是健康餐盒連鎖品牌，目前在雙北地區有8間連鎖門市，\r\n如果你對健康或餐飲連鎖產業有興...,//www.104.com.tw/job/7uajm?jobsource=jolist_b_...,台北市中正區,學歷不拘,其他餐飲業,經歷不拘,11~30人應徵,"月薪28,000~36,000元",員工10人


In [5]:
# 檔案輸出
Hualien_df.to_csv(os_path(["database", "104_database.csv"]), index= False)
error_df.to_csv(os_path(["error","104_crawler_error.csv"]), index = False)

# 測試區